# Late-fusion
Cargamos los datos de nuestras redes ya entrenadas



In [1]:
from google.colab import files
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import numpy as np

# Subir los archivos necesarios
print("Sube tus modelos y archivos .npy")
uploaded = files.upload()

# Cargar los modelos preentrenados
model1 = load_model('modelo1.h5')  # Modelo tabular
model2 = load_model('modelo2_resnet50.h5')  # ResNet50 o similar

# Cargar datos
X_train_scaled = np.load('X_train_scaled.npy')
X_val_scaled = np.load('X_val_scaled.npy')
X_test_scaled = np.load('X_test_scaled.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

# Cargar predicciones intermedias de ambos modelos
y_pred_val_model_a = np.load('y_pred_val_model_a.npy')  # Modelo 1 (tabular)
y_pred_test_model_a = np.load('y_pred_test_model_a.npy')
y_pred_val_model_b = np.load('y_pred_val_resnet.npy')  # Modelo 2 (ResNet50)
y_pred_test_model_b = np.load('y_pred_test_resnet.npy')

# Concatenar predicciones para la fusión tardía
X_train_fused = np.concatenate((y_pred_val_model_a, y_pred_val_model_b), axis=1)
X_val_fused = np.concatenate((y_pred_val_model_a, y_pred_val_model_b), axis=1)
X_test_fused = np.concatenate((y_pred_test_model_a, y_pred_test_model_b), axis=1)

# Definir el modelo final de fusión
input_fused = Input(shape=(X_train_fused.shape[1],))  # Tamaño de las características fusionadas
x = Dense(256, activation='relu')(input_fused)
x = Dense(128, activation='relu')(x)
final_output = Dense(3, activation='softmax')(x)  # 3 clases como ejemplo

final_model = Model(inputs=input_fused, outputs=final_output)

# Compilar el modelo
final_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Configurar callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('best_fusion_model.keras', monitor='val_loss', save_best_only=True)

# Entrenar el modelo de fusión
final_model.fit(
    X_train_fused, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_fused, y_val),
    callbacks=[early_stopping, reduce_lr, checkpoint]
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = final_model.evaluate(X_test_fused, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Guardar el modelo final
final_model.save("final_fusion_model.h5")
print("Modelo de fusión guardado como 'final_fusion_model.h5'.")



Sube tus modelos y archivos .npy


Saving X_train_scaled.npy to X_train_scaled.npy
Saving X_test_scaled.npy to X_test_scaled.npy
Saving X_val_scaled.npy to X_val_scaled.npy
Saving y_pred_test_model_a.npy to y_pred_test_model_a.npy
Saving y_pred_val_model_a.npy to y_pred_val_model_a.npy
Saving y_test.npy to y_test.npy
Saving y_train.npy to y_train.npy
Saving y_val.npy to y_val.npy


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modelo1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

Qué aspectos hay que tener en cuenta?

1. Aquí hemos empleado modelos pre-entrenados. Es eso un requisito obligatorio?

2. Qué representan las "features" del último modelo? Qué información encapsulan?

3. Podríamos reemplazar `final_classifier` por un SVM procedente de `sklearn`?

4. En qué situaciones este esquema es especialmente ventajoso, y por qué.

# Early-fusion

In [ ]:
from google.colab import files
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Dense, concatenate, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np

# Subir los archivos necesarios
print("Sube tus modelos y archivos .npy")
uploaded = files.upload()

# Cargar los modelos preentrenados
model1 = load_model('modelo1.h5')  # Modelo tabular
model2 = load_model('modelo2_resnet50.h5')  # ResNet50 o similar

# Ajustar los modelos para que actúen como extractores de características
model1_extractor = Model(inputs=model1.input, outputs=model1.layers[-2].output)  # Penúltima capa
model2_extractor = Model(inputs=model2.input, outputs=model2.layers[-2].output)

# Cargar datos preprocesados
X_train_scaled = np.load('X_train_scaled.npy')  # Para el modelo tabular
X_val_scaled = np.load('X_val_scaled.npy')
X_test_scaled = np.load('X_test_scaled.npy')
X_train_images = np.load('X_train_images.npy')  # Para el modelo de imágenes
X_val_images = np.load('X_val_images.npy')
X_test_images = np.load('X_test_images.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')
y_test = np.load('y_test.npy')

# Extraer características con ambos modelos
features_train_model1 = model1_extractor.predict(X_train_scaled)
features_val_model1 = model1_extractor.predict(X_val_scaled)
features_test_model1 = model1_extractor.predict(X_test_scaled)

features_train_model2 = model2_extractor.predict(X_train_images)
features_val_model2 = model2_extractor.predict(X_val_images)
features_test_model2 = model2_extractor.predict(X_test_images)

# Concatenar características para early fusion
X_train_fused = np.concatenate((features_train_model1, features_train_model2), axis=1)
X_val_fused = np.concatenate((features_val_model1, features_val_model2), axis=1)
X_test_fused = np.concatenate((features_test_model1, features_test_model2), axis=1)

# Construir el clasificador final
input_fused = Input(shape=(X_train_fused.shape[1],))
x = Dense(256, activation='relu')(input_fused)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
final_output = Dense(3, activation='softmax')(x)  # Cambia a tu número de clases

final_classifier = Model(inputs=input_fused, outputs=final_output)

# Compilar el modelo
final_classifier.compile(optimizer=Adam(learning_rate=0.001),
                         loss='sparse_categorical_crossentropy',
                         metrics=['accuracy'])

# Entrenar el clasificador final
final_classifier.fit(
    X_train_fused, y_train,
    validation_data=(X_val_fused, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-6),
        tf.keras.callbacks.ModelCheckpoint('best_early_fusion_model.h5', save_best_only=True)
    ]
)

# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = final_classifier.evaluate(X_test_fused, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Guardar el modelo final
final_classifier.save("early_fusion_classifier.h5")
print("Modelo de fusión temprana guardado como 'early_fusion_classifier.h5'.")


torch.Size([1, 512]) torch.Size([1, 25088])
Fused output shape: torch.Size([1, 25600])
Final prediction shape: torch.Size([1, 10])
